### Dense General Arbitrary Order Tensor Completion 
#### Author: Xiaoxiao Wu

In [1]:
import ctf,random
import numpy as np
import numpy.linalg as la
from ctf import random as crandom
glob_comm = ctf.comm()
import string
import operator
import functools as ft

#### Test Cases

In [2]:
class UnitTests:
        
    def test_general_purturb1(self):
        n = random.randint(2,5) # arbitrary dimension
        r = 1
        sparsity = .2
        regParam = 2
    
        # arbitrary tensor
        T,dimensions = init_tensor(n)
        factors = init_factors(dimensions,r)
       
        S = string.ascii_lowercase
        S = S[:len(dimensions)]
        
        tmp=ft.reduce(operator.mul,[ff.i(ss+'u') for (ff,ss) in zip(factors,S)],1)
        T.i(S) << tmp        

        omega = updateOmega(dimensions,sparsity)
        
        # purturb the first factor matrix
        factors[0] += crandom.random((dimensions[0],r))*.01
        others = factors.copy()
        others.pop(0)
        other_dimensions = dimensions.copy()
        other_dimensions.pop(0)
        nU = updateFactor(T,factors[0],others,regParam,omega,other_dimensions,r,S,0)
        
        nT = ctf.tensor(tuple(dimensions))        
        tmp=ft.reduce(operator.mul,[ff.i(ss+'u') for (ff,ss) in zip(factors,S)],nU.i('au'))
        nT.i(S) << tmp    
        
        assert(ctf.all(ctf.abs(nT - T < 1e-10)))
        print("passed test: test_general_purturb1")
            
    
    def test_general_purturbn(self):
        n = random.randint(2,5) # arbitrary dimension
        r = 1
        sparsity = .2
        regParam = 2
    
        # arbitrary tensor
        T,dimensions = init_tensor(n)
        factors = init_factors(dimensions,r)
    
        S = string.ascii_lowercase
        S = S[:len(dimensions)]      
        
        tmp=ft.reduce(operator.mul,[ff.i(ss+'u') for (ff,ss) in zip(factors,S)],1)
        T.i(S) << tmp    

        omega = updateOmega(dimensions,sparsity)

        # purturb the all n factor matrixces
        for i in range(len(factors)):
            factors[i] += crandom.random((dimensions[i],r))*.01
        
        others = factors.copy()
        others.pop(0)
        other_dimensions = dimensions.copy()
        other_dimensions.pop(0)
        nU = updateFactor(T,factors[0],others,regParam,omega,other_dimensions,r,S,0)
        
        nT = ctf.tensor(tuple(dimensions))   
        tmp=ft.reduce(operator.mul,[ff.i(ss+'u') for (ff,ss) in zip(factors,S)],nU.i('au'))
        nT.i(S) << tmp  
        
        assert(ctf.all(ctf.abs(nT - T < 1e-10)))
        print("passed test: test_general_purturbn")
            
        
    def runAllTests(self):
        self.test_general_purturb1()
        self.test_general_purturbn()

In [3]:
def updateFactor(T,curr,others,regParam,omega,other_dimensions,r,s,idx):
    '''Update single factor matrix by using the formula'''
    
    other_dimensions_ = other_dimensions.copy()
    other_dimensions_.append(r)
    M = ctf.tensor((tuple(other_dimensions_)))
    
    S = s[:idx] + s[idx + 1:]    
    temp=ft.reduce(operator.mul,[oo.i(ss+'u') for (oo,ss) in zip(others,S)],1)
    M.i(S+'u') << temp        
    
    D = np.prod(other_dimensions)
    [U_,S_,V_] = ctf.svd(M.reshape((D,r)))
    S_ = S_/(S_*S_ + regParam*ctf.ones(r))
    curr.set_zero()
    curr.i(s[idx]+'u') << V_.i("vu")*S_.i("v")*U_.reshape((tuple(other_dimensions_))).i(S+'v')*T.i(s)
    #curr *= normalize(curr,r)
    
    return curr

In [4]:
def getALSCtf(T,factors,regParam,omega,dimensions,r):
    '''
    Tensor completion using ALS
    '''
    it = 0
    E = ctf.tensor(tuple(dimensions))
    curr_err_norm = 0
    
    s = string.ascii_lowercase
    s = s[:len(dimensions)]
        
    e=ft.reduce(operator.mul,[ff.i(ss+'u') for (ff,ss) in zip(factors,s)],1)
    E.i(s) << T.i(s) - e
    curr_err_norm = sum([ctf.vecnorm(factors[idx]) for idx in range(len(factors))])
    curr_err_norm = ctf.vecnorm(E) + curr_err_norm*regParam
    
    while True:
        for idx in range(len(factors)):
            others = factors.copy()
            others.pop(idx)
            other_dimensions = dimensions.copy()
            other_dimensions.pop(idx)
        
            factors[idx] = updateFactor(T,factors[idx],others,regParam,omega,other_dimensions,r,s,idx)

        E.set_zero()
        next_err_norm = 0
        ne=ft.reduce(operator.mul,[ff.i(ss+'u') for (ff,ss) in zip(factors,s)],1)
        E.i(s) << T.i(s) - ne
        next_err_norm = sum([ctf.vecnorm(factors[idx]) for idx in range(len(factors))])
        next_err_norm = ctf.vecnorm(E) + next_err_norm*regParam
  
        if abs(curr_err_norm - next_err_norm) < .001 or it > 10:
            break
         
        print(curr_err_norm, next_err_norm)
        curr_err_norm = next_err_norm
        it += 1
    
    print("Number of iterations: ", it)
    return factors

In [5]:
def normalize(Z,r):
    norms = ctf.tensor(r)
    norms.i("u") << Z.i("pu")*Z.i("pu")
    norms = 1./norms**.5
    X = ctf.tensor(copy=Z)
    Z.set_zero()
    Z.i("pu") << X.i("pu")*norms.i("u")
    return 1./norms

In [6]:
def init_tensor(n):

    dimensions=[random.randint(1,10) for _ in range(n)]  
    T = ctf.tensor(tuple(dimensions))

    return T,dimensions


def init_factors(dimensions,r):
    
    factors = [ctf.random.random((dd,r)) for dd in dimensions]    
    
    return factors


def updateOmega(dimensions,sparsity):
    '''
    Gets a random subset of rows
    '''
    Actf = ctf.tensor(tuple(dimensions),sp=True)
    Actf.fill_sp_random(1,1,sparsity)
    #omegactf = ((Actf > 0)*ctf.astensor(1.))
    
    return Actf

In [7]:
def main():
    
    ut = UnitTests()
    ut.runAllTests()
    
    n = random.randint(2,5) # arbitrary dimension
    r = 1
    sparsity = .2
    regParam = 20
    
    # arbitrary tensor
    T,dimensions = init_tensor(n)
    print("dimensions: ", tuple(dimensions))
    factors = init_factors(dimensions,r)
            
    s_tmp = string.ascii_lowercase
    s_tmp = s_tmp[:len(dimensions)]
    
    tmp=ft.reduce(operator.mul,[ff.i(ss+'u') for (ff,ss) in zip(factors,s_tmp)],1)
    T.i(s_tmp) << tmp 
    
    omega = updateOmega(dimensions,sparsity)
        
    getALSCtf(T,factors,regParam,omega,dimensions,r)
    

In [8]:
main()

passed test: test_general_purturb1
passed test: test_general_purturbn
dimensions:  (2, 3, 8, 5)
77.4761811427053 0.5967783694513268
0.5967783694513268 0.37335892172227175
Number of iterations:  2
